In [2]:
import csv
import random
import string
import names
from datetime import *

In [47]:
def gen_datetime(min_year=1900, max_year=datetime.now().year):
# generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
# start = datetime(min_year, 1, 1, 00, 00, 00)
    #start = date(1999, 12, 11)
    #years = max_year - min_year + 1
    #end = start + timedelta(days=365 * years)
    #return start + (end - start) * random.random()
    
    return str(random.randint(min_year, max_year)) + "-" + str(random.randint(1,13)) + "-" + str(random.randint(1,28))

In [59]:
def gen_customers(num):
    result = []
    for i in range(1,num+1):
        result.append([
                       i,                      # NFC_ID
                       names.get_first_name(), # First name
                       names.get_last_name(),  # Last name
                       str(gen_datetime()),
                       random.choice(string.ascii_letters) + random.choice(string.ascii_letters) + str(random.randint(100000, 999999)),
                       random.choice(["ID", "Passport"]),
                       "Authority" + str(random.randint(1,11))
                      ])
    return result

In [58]:
customers = gen_customers(100)
for c in customers:
    print(c)

[1, 'Deidre', 'Lewis', '1967-7-24', 'ny233101', 'ID', 'Authority10']
[2, 'Oren', 'Stoehr', '1919-13-4', 'Pa292291', 'Passport', 'Authority11']
[3, 'Lillie', 'Garcia', '1971-13-16', 'kb194646', 'Passport', 'Authority4']
[4, 'Lynn', 'Gonzalez', '1958-8-18', 'UP887430', 'ID', 'Authority2']
[5, 'Houston', 'Kennedy', '1905-7-28', 'Td536463', 'ID', 'Authority10']
[6, 'Heather', 'Garcia', '1919-8-15', 'Ir375185', 'ID', 'Authority1']
[7, 'Timothy', 'Oconnor', '1921-8-4', 'bt634195', 'ID', 'Authority10']
[8, 'Robert', 'Hardy', '1903-3-27', 'qq361650', 'Passport', 'Authority6']
[9, 'Nina', 'Stoltzfus', '1960-4-13', 'to450567', 'ID', 'Authority4']
[10, 'Steve', 'Toribio', '1979-6-17', 'ur207723', 'ID', 'Authority3']
[11, 'Lorri', 'Suennen', '2016-13-4', 'GR124084', 'Passport', 'Authority10']
[12, 'Linda', 'Toro', '1976-13-11', 'Md858744', 'Passport', 'Authority10']
[13, 'Timothy', 'Martelli', '1951-8-14', 'HU406013', 'ID', 'Authority9']
[14, 'William', 'Wilson', '1976-2-22', 'lr395337', 'ID', 'Au

In [5]:
def random_date(min_year=1900, max_year=datetime.now().year, mode = 'date'):
    if(mode == 'date'):
        start = date(min_year, 1, 1)
    elif(mode == 'datetime'):
        start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    if(mode == 'date'):
        return str(start + (end - start) * random.random())
    elif(mode == 'datetime'):
        return str(start + (end - start) * random.random())[:-7]
        

def random_string(strlen, chars = string.ascii_letters, mode = 'var', repeat = True):
    if (mode == 'date'):
        return random_date()
    elif (mode == 'datetime'):
        return random_date(mode = 'datetime')
    elif(mode == 'var'):
        strlen = random.randint(1,strlen)
    if(repeat):
        return ''.join(random.choice(chars) for i in range (random.randint(1,strlen)))
    else:
        return ''.join(random.sample(chars, strlen))
    

def random_row(rowlen, strlen, chars, mode, repeat): #strlen, chars, mode are lists
    return [random_string(strlen[i], chars[i], mode[i], repeat[i]) for i in range(rowlen)]

#attr_names: βάζεις λίστα των χαρακτηριστικών
#strlen: λίστα μηκών των πεδίων
#chars: string με όλους τους χαρακτήρες που μπορούν να χρησιμοποιηθούν για την παραγωγή
#mode: var όταν το strlen δηλώνει μέγιστο μήκος, fix αλλιώς. Για ημερομηνίες βάζεις 'date', 'datetime'
#repeat: Ήθελα να γίνεται το εξής: True-> μοναδικές τιμές, False: μπορούν να επαναλαμβάνονται, αλλά το σκάτωσα
#size: πλήθος tuples

def random_table(attr_names, strlen, chars, mode, repeat, size):
    res = [random_row(len(attr_names), strlen, chars, mode, repeat) for i in range (size)]
    res.insert(0, attr_names)
    return res

letters = string.ascii_letters
digits = string.digits
allchars = letters+digits

print(random_table(['nfc_id', 'name', 'surname', 'date_of_birth', 'id_document_number', 'id_document_type', 
                    'id_document_authority'], 
                   [5, 31, 31, 10, 20, 31, 31],
                   [digits, letters, letters, letters, letters, letters, letters],
                   ['var', 'var', 'var', 'date', 'var', 'var', 'var'],
                   [False, True, True, True, True, True, True],
                   50))

# with open('Customers.csv', 'w') as csv_file:
#     csv_writer = csv.writer(csv_file)
    

[['nfc_id', 'name', 'surname', 'date_of_birth', 'id_document_number', 'id_document_type', 'id_document_authority'], ['378', 'F', 'onnZ', '1970-08-07', 'RFUEMqqeftdVNHrm', 'e', 'X'], ['940', 'dVrkL', 'c', '2003-06-21', 'iT', 'O', 'C'], ['5', 'xtGsoujmOvwJgQLcIlkQUgPjLDowWn', 'J', '2006-07-17', 'dj', 'MIFd', 'iJiq'], ['94', 'l', 'J', '1918-07-17', 'HAODbD', 'WyJgHdMWTuATRR', 'pa'], ['6', 'Hpu', 'WBArkiQeDCicoscv', '1958-07-27', 'Pzau', 'tM', 'PfvvdKYvXZtRqKlglc'], ['4', 'fFjUVQgN', 'gKPEg', '1985-08-31', 'w', 'gVLlRvUA', 'X'], ['74', 'VqQZMrHEYpDmnLuRLXNNjU', 'BBfHgvCX', '1911-07-26', 'nRujB', 'VBPYVdBEtrrvVFTXRM', 'nAToYIb'], ['5', 'whhsiVEJuBKzfWBySCYRWu', 'mTts', '1995-06-17', 'xeAHtvrwVRrqs', 'GZr', 'unnnlNBpfFyyh'], ['48790', 'BnZbWjzhsQlUd', 'tXZPlCfyWzW', '1915-05-02', 'CQYMkffT', 'Edd', 'Fwn'], ['0328', 'TQnneWNsfWeweJvXlR', 'Y', '2015-02-03', 'rgvdrgCMOCYYc', 'wphkaBEDxSqyuujSRO', 'UAglttosYRffYNBdjZAXfKtbzsf'], ['6', 'rwrfXNELiQ', 'eJVgcRXamESCJsM', '1901-10-15', 'b', 'IIwinb',